# Mean Field Inference

In [1]:
import numpy as np
from mlxtend.data import loadlocal_mnist
import matplotlib.pyplot as plt


In [2]:
# load MNIST images

ims, _ = loadlocal_mnist(
        images_path='train-images.idx3-ubyte', 
        labels_path='train-labels.idx1-ubyte')


In [3]:
# find 8 neighbors of a given pixel

def find_neighbor(im, idx, w, h):
    s = w*h
    nbr = []
    if idx < 0 or idx > s-1:
        return nbr
    if idx-1 >= 0 and idx%w != 0:
        nbr.append(idx-1)
    if idx+1 < s and (idx+1)%w != 0:
        nbr.append(idx+1)
    if idx-w >= 0:
        nbr.append(idx-w)
    if idx+w < s:
        nbr.append(idx+w)
    if idx-w-1 >= 0 and idx%w != 0:
        nbr.append(idx-w-1)
    if idx-w+1 >= 0 and (idx+1)%w != 0:
        nbr.append(idx-w+1)
    if idx+w-1 < s and idx%w != 0:
        nbr.append(idx+w-1)
    if idx+w+1 < s and (idx+1)%w != 0:
        nbr.append(idx+w+1)
    return nbr


In [4]:
# Update an image using mean field inference until it converges(update<delta)

def update_image(im, theta_h, theta_x, pi_i, delta):
    temp_im = im.copy()
    temp_pi_i = pi_i.copy()
    diff = 0
    for p in range(im.shape[0]):
        nbr = find_neighbor(im, p, 28, 28)
        q_plus = 0
        q_minus = 0
        for n in nbr:
            q_plus += ((theta_h) * (2*pi_i[n] - 1) + theta_x * im[n])
            q_minus += -q_plus
        temp_pi_i[p] = np.exp(q_plus)/(np.exp(q_plus)+np.exp(q_minus))
        if abs(temp_pi_i[p] - pi_i[p]) > diff:
            diff = abs(temp_pi_i[p] - pi_i[p])
        if temp_pi_i[p] > 0.5:
            temp_im[p] = 1
        else:
            temp_im[p] = -1
    im = temp_im
    pi_i = temp_pi_i
    if diff > delta:
        update_image(im, theta_h, theta_x, pi_i, delta)
    return im, pi_i


In [5]:
# take the first 500 MNIST images as original images and denoise 2% pixels randomly

im_in = ims[0:500][:]/255
im_noise = im_in.copy()
rand_idx = np.arange(im_in.shape[1])
for i in range(im_in.shape[0]):
    for j in range(im_in.shape[1]):
        if im_in[i][j]>=0.5:
            im_in[i][j] = 1
            im_noise[i][j] = 1
        else:
            im_in[i][j] = -1
            im_noise[i][j] = -1
    np.random.shuffle(rand_idx)
    for k in range(int(len(rand_idx)*0.02)):
        p = rand_idx[k]
        im_noise[i][p]*=-1
        

In [6]:
# reconstruct images

pi = np.ones((im_in.shape[0], im_in.shape[1]))*0.5
im_out = im_noise.copy()
delta = 0.001
for i in range(im_out.shape[0]):
    im_out[i], pi[i]= update_image(im_out[i], 0.2, 0.2, pi[i], delta)

KeyboardInterrupt: 

In [ ]:
# plot original, noised, denoised images
# replace this image to be most&least accurate reconstructions    
fig=plt.figure(figsize=(8, 8))
columns = 3
rows = 10
for i in range(10):
    plt.figure()
    plt.imshow(im_in[i].reshape(28,28), cmap="gray")
    plt.figure()
    plt.imshow(im_noise[i].reshape(28,28), cmap="gray")
    plt.figure()
    plt.imshow(im_out[i].reshape(28,28), cmap="gray")
plt.show() 

In [ ]:
### TO DO: find the most&least accurate reconstructions and plot original, noised, denoised images
def get_accuracy(test):
    total_accuracy = np.zeros(im_in.shape[0])
    for i in range(im_in.shape[0]):
        wrong = 0
        curr = im_in[i]
        compare = test[i]
        for j in range(curr.shape[0]):
            if curr[j] != compare[j]:
                   wrong += 1
        accuracy = 1 - 1.0 * (wrong)/curr.shape[0]
        total_accuracy[i] = accuracy
    average_accuracy = sum(total_accuracy)/500
    print(average_accuracy)
    return average_accuracy

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==-1:
           TN += 1
        if y_hat[i]==-1 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
parameters = [0, 0.2, 1, 2]

pred = {}
pi = np.ones((im_in.shape[0], im_in.shape[1]))*0.5
im_out = im_noise.copy()
delta = 0.001
for para in parameters:
    for i in range(im_out.shape[0]):
        im_out[i], pi[i]= update_image(im_out[i], para, 0.2, pi[i], delta)
    pred[para] = im_out.reshape(500 * 784)
label = im_in.reshape(500 * 784)
TP, FP, TN, FN = perf_measure(label, pred[para])
print(TP,FP,TN,FN)

In [ ]:
pi = np.ones((im_in.shape[0], im_in.shape[1]))*0.5
im_out = im_noise.copy()
delta = 0.001
run_time = 0
for i in range(im_out.shape[0]):
    im_out[i], pi[i]= update_image(im_out[i], -1, 0.2, pi[i], delta, run_time)

In [ ]:
### TO DO: plot receiving operating curve -1, 0, 0.2, 1, 2
from sklearn import metrics

parameters = [-1, 0, 0.2, 1, 2]

pred = {}

label = im_in.reshape(500 * 784)


for para in parameters:
    fpr = dict()
    tpr = dict()
    pi = np.ones((im_in.shape[0], im_in.shape[1]))*0.5
    im_out = im_noise.copy()
    delta = 0.001
    for i in range(im_out.shape[0]):
        im_out[i], pi[i]= update_image(im_out[i], para, 0.2, pi[i], delta)
    pred[para] = im_out.reshape(500 * 784)
    fpr[para],tpr[para], _ = metrics.roc_curve(label, pred[para])
    plt.scatter(fpr[para], tpr[para])
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title("ROC Curve")
    plt.show()
    